### Library Import

In [156]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
import numpy as np
! pip install folium==0.5.0
import folium
print("Success!")

Success!


In [157]:
# The code was removed by Watson Studio for sharing.

### Data Import

In [158]:
# specify URL to Neighborhood data on Github

url = "https://raw.githubusercontent.com/xiaolong89/practice/master/Las%20Vegas%20Data.csv"
data = pd.read_csv(url)
print("The dataframe has", data.shape[0], "rows.")
data.head()

The dataframe has 61 rows.


,City,zipCodes,Neighborhood
0,Las Vegas,89166,Centennial
1,Las Vegas,89143,Centennial
2,Las Vegas,89131,Centennial
3,Las Vegas,89149,Centennial
4,Las Vegas,89130,Centennial


In [159]:
# gather lat/lng coordinates

coord_url = "https://raw.githubusercontent.com/xiaolong89/practice/master/NV%20zip%20code%20data.csv"
coord = pd.read_csv(coord_url)
coord.rename(columns = {'Zip':'zipCodes'}, inplace = True)
coord.drop(["City", "State"], axis = 1, inplace = True)
coord.head()

,zipCodes,Latitude,Longitude
0,89570,40.541218,-119.586934
1,89835,41.208288,-114.860980
2,89148,36.049095,-115.284850
3,89147,36.113059,-115.273430
4,89599,40.541218,-119.586934


### Data Cleaning

In [160]:
# merge the "data" dataframe and "coord" dataframe to have one dataframe with necessary data for analysis

las_vegas = data.merge(coord.set_index("zipCodes"), on = "zipCodes")
las_vegas.head(15)

,City,zipCodes,Neighborhood,Latitude,Longitude
0,Las Vegas,89143,Centennial,36.326315,-115.25788
1,Las Vegas,89131,Centennial,36.294596,-115.24685
2,Las Vegas,89149,Centennial,36.266266,-115.28656
3,Las Vegas,89130,Centennial,36.257531,-115.22728
4,Las Vegas,89130,Summerlin,36.257531,-115.22728
5,Las Vegas,89129,Summerlin,36.231139,-115.28461
6,Las Vegas,89134,Summerlin,36.204548,-115.31207
7,Las Vegas,89128,Summerlin,36.193501,-115.26500
8,Las Vegas,89135,South Summerlin,36.131341,-115.32331
9,Las Vegas,89117,South Summerlin,36.140974,-115.28205


### Explore and Cluster Neighborhoods

In [161]:
# Las Vegas geo coordinates

address = 'Las Vegas, NV'

geolocator = Nominatim(user_agent = "vegas_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Las Vegas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Las Vegas are 36.1672559, -115.1485163.


In [162]:
# create map of Las Vegas

vegas_map = folium.Map(location = [latitude, longitude], zoom_start = 10)
vegas_map

In [163]:
# plot zipCodes on vegas_map

for lat, lng, code, neighborhood in zip(
        las_vegas['Latitude'],
        las_vegas['Longitude'],
        las_vegas['zipCodes'],
        las_vegas['Neighborhood']):
    label = '{}, {}'.format(neighborhood, code)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(vegas_map)  
vegas_map

In [164]:
# examine the South Summerlin neighborhood from the dataframe

neighborhood_name = las_vegas.loc[8, 'Neighborhood']
print("The neighborhood's name is", neighborhood_name + ".")

The neighborhood's name is South Summerlin.


In [165]:
# Get neighborhood's lat/lng coordinates

neighborhood_lat = las_vegas.loc[8, 'Latitude'] 
neighborhood_lng = las_vegas.loc[8, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_lng))

Latitude and longitude values of South Summerlin are 36.131341, -115.32331.


In [166]:
# Get top 100 venues w/radius of 500 meters

LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lng, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [167]:
# discover the category of each venue

def category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [168]:
# clean JSON and convert to dataframe

venues = results['response']['groups'][0]['items']

nearby_venues = pd.json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Planet Earth,Vineyard,36.131454,-115.321503
1,Las Vegas Farmer's Market,Farmers Market,36.131491,-115.321607
2,Wimbledon For Locals,Tennis Court,36.126859,-115.323550


In [169]:
# analyze all Las Vegas neighborhoods using process in previous cell

def nearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [170]:
las_vegas_venues = nearbyVenues(names = las_vegas['Neighborhood'],
                                   latitudes = las_vegas['Latitude'],
                                   longitudes = las_vegas['Longitude']
                                  )
las_vegas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centennial,36.266266,-115.28656,Sls Stables,36.264492,-115.285064,Stables
1,Centennial,36.257531,-115.22728,Ice Cream Truck,36.257172,-115.230457,Food Truck
2,Centennial,36.257531,-115.22728,Polar Bear Diving,36.255035,-115.224011,Pool
3,Centennial,36.257531,-115.22728,Photography By Rebecca Wood,36.257442,-115.222756,Art Gallery
4,Centennial,36.257531,-115.22728,Fresh And Easy,36.254937,-115.223574,Grocery Store


In [171]:
# find number of venues for each neighborhood

las_vegas_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anthem,33,33,33,33,33,33
Boulder City,2,2,2,2,2,2
Centennial,5,5,5,5,5,5
Downtown,14,14,14,14,14,14
Green Valley / Henderson,22,22,22,22,22,22
North Las Vegas,20,20,20,20,20,20
Pahrump,3,3,3,3,3,3
South Summerlin,30,30,30,30,30,30
Southeast,26,26,26,26,26,26


In [172]:
# number of unique categories from all venues

print("There are {} unique venue categories".format(len(las_vegas_venues["Venue Category"].unique())))

There are 129 unique venue categories


In [173]:
# Neighborhood analysis

# one hot encoding
las_vegas_onehot = pd.get_dummies(las_vegas_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
las_vegas_onehot['Neighborhood'] = las_vegas_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [las_vegas_onehot.columns[-1]] + list(las_vegas_onehot.columns[:-1])
las_vegas_onehot = las_vegas_onehot[fixed_columns]

las_vegas_onehot.head()

,Neighborhood,ATM,Airport,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Tree,Vegetarian / Vegan Restaurant,Video Store,Vineyard,Weight Loss Center,Yoga Studio
0,Centennial,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centennial,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centennial,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centennial,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centennial,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
# group rows by neighborhood and review mean of frequency of occurrence of each cat

las_vegas_grouped = las_vegas_onehot.groupby('Neighborhood').mean().reset_index()
las_vegas_grouped.head()

,Neighborhood,ATM,Airport,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Tree,Vegetarian / Vegan Restaurant,Video Store,Vineyard,Weight Loss Center,Yoga Studio
0,Anthem,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.030303,0.030303,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,Boulder City,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Centennial,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Downtown,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Green Valley / Henderson,0.045455,0.0,0.0,0.0,0.0,0.0,0.045455,0.000000,0.000000,...,0.045455,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0


In [175]:
# find top 10 most common venues in each neighborhood

def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe

neighborhoods_venues = pd.DataFrame(columns = columns)
neighborhoods_venues['Neighborhood'] = las_vegas_grouped['Neighborhood']

for ind in np.arange(las_vegas_grouped.shape[0]):
    neighborhoods_venues.iloc[ind, 1:] = most_common_venues(las_vegas_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anthem,Italian Restaurant,Spa,Business Service,Burger Joint,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Butcher,Pizza Place,Pharmacy
1,Boulder City,Baseball Field,Athletics & Sports,Electronics Store,Fast Food Restaurant,Farmers Market,Farm,Event Service,Ethiopian Restaurant,Duty-free Shop,Fried Chicken Joint
2,Centennial,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
3,Downtown,Convenience Store,Bakery,Construction & Landscaping,Chinese Restaurant,Pool,Rental Car Location,Nightclub,Farm,Outdoor Supply Store,Speakeasy
4,Green Valley / Henderson,Fast Food Restaurant,Park,Pool,Gym,Grocery Store,Athletics & Sports,Golf Course,Casino,Discount Store,Pizza Place


In [176]:
# Cluster the neighborhoods

kclusters = 5

las_vegas_grouped_cluster = las_vegas_grouped.drop('Neighborhood', 1)

# run k-means clustering

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(las_vegas_grouped_cluster)

# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10]

array([0, 1, 3, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [179]:
# new dataframe w/cluster and top 10 venues

neighborhoods_venues.insert(0, "Cluster Labels", kmeans.labels_)

las_vegas_merged = las_vegas

# merge las_vegas_grouped with las_vegas to add lat/lng

las_vegas_merged = las_vegas_merged.join(neighborhoods_venues.set_index('Neighborhood'), on = 'Neighborhood')

las_vegas_merged.head()

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Las Vegas,89143,Centennial,36.326315,-115.25788,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
1,Las Vegas,89131,Centennial,36.294596,-115.24685,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
2,Las Vegas,89149,Centennial,36.266266,-115.28656,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
3,Las Vegas,89130,Centennial,36.257531,-115.22728,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
4,Las Vegas,89130,Summerlin,36.257531,-115.22728,0,Italian Restaurant,Thai Restaurant,Bed & Breakfast,Optical Shop,Pool,Mexican Restaurant,Fast Food Restaurant,Snack Place,Food Truck,Bakery


In [180]:
# visualize the clusters

map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

# set color scheme for the clusters

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

markers_colors = []
for lat, lng, poi, cluster in zip(
        las_vegas_merged['Latitude'], 
        las_vegas_merged['Longitude'], 
        las_vegas_merged['Neighborhood'], 
        las_vegas_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster for top venues

#### Cluster 0

In [191]:
cluster0 = las_vegas_merged.loc[las_vegas_merged['Cluster Labels'] == 0]
cluster0

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Las Vegas,89130,Summerlin,36.257531,-115.227280,0,Italian Restaurant,Thai Restaurant,Bed & Breakfast,Optical Shop,Pool,Mexican Restaurant,Fast Food Restaurant,Snack Place,Food Truck,Bakery
5,Las Vegas,89129,Summerlin,36.231139,-115.284610,0,Italian Restaurant,Thai Restaurant,Bed & Breakfast,Optical Shop,Pool,Mexican Restaurant,Fast Food Restaurant,Snack Place,Food Truck,Bakery
6,Las Vegas,89134,Summerlin,36.204548,-115.312070,0,Italian Restaurant,Thai Restaurant,Bed & Breakfast,Optical Shop,Pool,Mexican Restaurant,Fast Food Restaurant,Snack Place,Food Truck,Bakery
7,Las Vegas,89128,Summerlin,36.193501,-115.265000,0,Italian Restaurant,Thai Restaurant,Bed & Breakfast,Optical Shop,Pool,Mexican Restaurant,Fast Food Restaurant,Snack Place,Food Truck,Bakery
8,Las Vegas,89135,South Summerlin,36.131341,-115.323310,0,Convenience Store,Grocery Store,Coffee Shop,Park,Pizza Place,ATM,Fast Food Restaurant,Concert Hall,Lounge,Bus Stop
9,Las Vegas,89117,South Summerlin,36.140974,-115.282050,0,Convenience Store,Grocery Store,Coffee Shop,Park,Pizza Place,ATM,Fast Food Restaurant,Concert Hall,Lounge,Bus Stop
10,Las Vegas,89145,South Summerlin,36.167731,-115.267910,0,Convenience Store,Grocery Store,Coffee Shop,Park,Pizza Place,ATM,Fast Food Restaurant,Concert Hall,Lounge,Bus Stop
11,Las Vegas,89144,South Summerlin,36.181142,-115.325570,0,Convenience Store,Grocery Store,Coffee Shop,Park,Pizza Place,ATM,Fast Food Restaurant,Concert Hall,Lounge,Bus Stop
12,Las Vegas,89102,Spring Valley,36.145303,-115.183580,0,Convenience Store,Rental Car Location,Fast Food Restaurant,Breakfast Spot,Pizza Place,Restaurant,Korean Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office
13,Las Vegas,89146,Spring Valley,36.141119,-115.224930,0,Convenience Store,Rental Car Location,Fast Food Restaurant,Breakfast Spot,Pizza Place,Restaurant,Korean Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office


#### Cluster 1

In [190]:
cluster1 = las_vegas_merged.loc[las_vegas_merged['Cluster Labels'] == 1]
cluster1

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Las Vegas,89005,Boulder City,35.968615,-114.83354,1,Baseball Field,Athletics & Sports,Electronics Store,Fast Food Restaurant,Farmers Market,Farm,Event Service,Ethiopian Restaurant,Duty-free Shop,Fried Chicken Joint


#### Cluster 2

In [189]:
cluster2 = las_vegas_merged.loc[las_vegas_merged['Cluster Labels'] == 2]
cluster2

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Las Vegas,89122,Whitney,36.105240,-115.04592,2,Intersection,Home Service,Dog Run,Men's Store,Massage Studio,Pizza Place,Gym / Fitness Center,Farm,Department Store,Dessert Shop
42,Las Vegas,89121,Whitney,36.121972,-115.09030,2,Intersection,Home Service,Dog Run,Men's Store,Massage Studio,Pizza Place,Gym / Fitness Center,Farm,Department Store,Dessert Shop
43,Las Vegas,89120,Whitney,36.087623,-115.08872,2,Intersection,Home Service,Dog Run,Men's Store,Massage Studio,Pizza Place,Gym / Fitness Center,Farm,Department Store,Dessert Shop


#### Cluster 3

In [188]:
cluster3 = las_vegas_merged.loc[las_vegas_merged['Cluster Labels'] == 3]
cluster3

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Las Vegas,89143,Centennial,36.326315,-115.25788,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
1,Las Vegas,89131,Centennial,36.294596,-115.24685,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
2,Las Vegas,89149,Centennial,36.266266,-115.28656,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market
3,Las Vegas,89130,Centennial,36.257531,-115.22728,3,Food Truck,Pool,Stables,Art Gallery,Grocery Store,Discount Store,Dive Bar,Dessert Shop,Doctor's Office,Farmers Market


#### Cluster 4

In [187]:
cluster4 = las_vegas_merged.loc[las_vegas_merged['Cluster Labels'] == 4]
cluster4

,City,zipCodes,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Las Vegas,89041,Pahrump,36.655797,-116.004795,4,Convenience Store,History Museum,Casino,Fast Food Restaurant,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
49,Las Vegas,89060,Pahrump,36.264185,-116.038857,4,Convenience Store,History Museum,Casino,Fast Food Restaurant,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
50,Las Vegas,89048,Pahrump,36.218341,-116.009940,4,Convenience Store,History Museum,Casino,Fast Food Restaurant,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
51,Las Vegas,89061,Pahrump,36.119299,-115.938992,4,Convenience Store,History Museum,Casino,Fast Food Restaurant,Dessert Shop,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
